# Simple reverse autodiff computational chain

- [Simple reverse-mode Autodiff in Julia - Computational Chain - YouTube](https://www.youtube.com/watch?v=ARBf1R0jm_M)
- <https://github.com/Ceyron/machine-learning-and-simulation/blob/main/english/adjoints_sensitivities_automatic_differentiation/simple_reverse_autodiff_computational_chain.jl>

In [1]:
f(x) = exp(sin(sin(x)))

f(2.0)

2.2013533791690376

In [2]:

∇f(x) = exp(sin(sin(x))) * cos(sin(x)) * cos(x)

∇f(2.0)

-0.562752038662712

In [3]:

(f(2.0 + 1e-8) - f(2.0)) / 1e-8

-0.5627520671680486

In [4]:
function backprop_rule(::typeof(sin), x)
    return sin(x), ∇x -> cos(x) * ∇x
end

backprop_rule (generic function with 1 method)

In [5]:

function backprop_rule(::typeof(exp), x)
    y = exp(x)
    return y, ∇x -> y * ∇x
end

backprop_rule (generic function with 2 methods)

In [6]:
# vjp = Vector Jacobian product

function vjp(chain, value)
    stack = []

    # Primal Pass
    for operation in chain
        value, pullback = backprop_rule(operation, value)
        push!(stack, pullback)
    end

    function pullback(cotangent)
        for back in reverse(stack)
            cotangent = back(cotangent)
        end
        return cotangent
    end

    return value, pullback
end

vjp (generic function with 1 method)

In [7]:
out, back = vjp([sin, sin, exp], 2.0)

(2.2013533791690376, var"#pullback#13"{Vector{Any}}(Any[var"#9#10"{Float64}(2.0), var"#9#10"{Float64}(0.9092974268256817), var"#11#12"{Float64}(2.2013533791690376)]))

In [8]:
back(1.0)

-0.562752038662712

In [9]:
function val_and_grad(chain, x)
    y, pullback = vjp(chain, x)
    derivative = pullback(1.0)
    return y, derivative
end

val_and_grad (generic function with 1 method)

In [10]:
val_and_grad([sin, sin, exp], 2.0)

(2.2013533791690376, -0.562752038662712)

In [11]:
f(2.0), ∇f(2.0)

(2.2013533791690376, -0.562752038662712)